In [12]:
__author__ = 'Carl Stubens <cstubens@noao.edu>'
__edited__ = 'Gautham Narayan <gnarayan@noao.edu>, Chien-Hsiu Lee <lee@noao.edu>'
__version__ = '20200825' # yyyymmdd
__datasets__ = ['']
__keywords__ = ['ANTARES', 'transient']

# ANTARES Filter Development Kit

_Carl Stubens, Gautham Narayan, Chien-Hsiu Lee, ANTARES Team._

_Many thanks to Mike Fitzpatrick, Adam Scott, Knut Olsen, Jennifer Andrews, Robert Nikutta._

## Summary

This Notebook demonstrates how to write filters for [ANTARES](http://antares.noirlab.edu) and test them against a sample of real data from [ZTF](http://ztf.caltech.edu/).

This Notebook is intended to be used in NOIRLab DataLab's Jupyter environment. There, you will have access to ANTARES data. If you're not running in DataLab, [sign up for DataLab](https://datalab.noao.edu), then [log in to the notebook server](https://datalab.noao.edu/devbooks).

For new Data Lab accounts, this notebook will be automatically included in your `notebooks/` directory. Otherwise, you can save this `.ipynb` notebook file locally, and then upload it to your Data Lab Jupyter notebook server (use the 'Upload' button in the upper right corner).

In Data Lab, you MUST use the Kernel version "Python 3 (ANTARES 0.4)".
## Goals

To demonstrate:

1. How to write filters using the ANTARES filter API.
1. How to test filters against a small test dataset.


## Table of Contents

* [0. Background information on ANTARES](#background)
* [1. Connect to ANTARES database](#connect)
* [2. Write a Filter](#write)
 * [2.1 Hellow World](#write-one)
 * [2.2 Example of a real Filter](#write-two)
 * [2.3 Structure of a Filter](#write-three)
* [3. Test a Filter](#test)
* [4. Upload and use Data File](#data)
* [5. Submit Filter to ANTARES](#submit)

<a class="anchor" id="background"></a>
## 0. Background information on ANTARES

ANTARES receives alerts from surveys in real-time and sends them through a processing pipeline. The pipeline contains the following stages:

1. Associate the Alert with the nearest point of known past measurements within a 1" radius. We call this a Locus.
2. Discard Alerts with a high probability of being false detections.
3. Discard Alerts with poor image quality.
4. Look up associated objects in our catalogs.
5. If the Alert's Locus has two or more measurements on it, execute the Filters.

The filters are python functions which take a LocusData object as a single parameter. Functions on the LocusData provide access to the Alert's properties, the data from past Alerts on the Locus, and the associated catalog objects. The LocusData also provides functions to set new properties on the Alert, and to send it to output streams.

<a class="anchor" id="connect"></a>
## 1. Initalize the dev kit

This will configure the `antares` package to connect to the test database

In [13]:
# Imports
import antares.devkit as dk
dk.init()
# You should see a happy message that says that "ANTARES DevKit is ready!"

Connecting to MySQL server...
Connecting to Cassandra cluster...
Connecting to ElasticSearch cluster...
Testing loading a random Locus with `dk.get_locus()`...

ANTARES v0.4.0 DevKit is ready!
Website: http://antares.noirlab.edu
Documentation: http://noao.gitlab.io/antares/filter-documentation/



<a class="anchor" id="write"></a>
## 2. Write a Filter

<a class="anchor" id="write-one"></a>
### 2.1 Hello world

Let’s make a simple `HelloWorld` filter which tags all Loci `hello_world`:


In [3]:
class HelloWorld(dk.Filter):
    OUTPUT_TAGS = [
        {
            'name': 'hello_world',
            'description': 'hello!',
        },
    ]

    def run(self, locus):
        print('Hello Locus', locus.locus_id)
        locus.tag('hello_world')

Let’s run the filter on a randomly chosen real Locus from the database:

In [4]:
# Fetch 1 random Locus ID from the test dataset
locus_id = dk.get_locus_ids(1)[0]

# Execute HelloWorld filter on the locus
report = dk.run_filter(HelloWorld, locus=locus_id)

# `run_filter()` returns a report of what the filter did. Take a look at it:
print(report)

Hello Locus ANT2018cnt6o
{'locus_id': 'ANT2018cnt6o', 'locus_data': LocusDataAPI(locus_id="ANT2018cnt6o"), 't': 0.0008563995361328125, 'new_locus_properties': {}, 'new_alert_properties': {}, 'new_tags': ['hello_world'], 'raised_halt': False}


<a class="anchor" id="write-two"></a>
### 2.2 Example of a real Filter

As an example, here’s a version of the “High SNR” filter which is one of the defaults included in ANTARES. It tags Loci which have at least one Alert with a high signal-noise ratio:

In [5]:
class HighSNR(dk.Filter):
    NAME = "High SNR"
    ERROR_SLACK_CHANNEL = ""  # Put your Slack user ID here
    REQUIRED_LOCUS_PROPERTIES = [
        'ztf_object_id',
    ]
    REQUIRED_ALERT_PROPERTIES = [
        'passband',
        'ztf_sigmapsf',
    ]
    OUTPUT_LOCUS_PROPERTIES = []
    OUTPUT_ALERT_PROPERTIES = []
    OUTPUT_TAGS = [
        {
            'name': 'high_snr',
            'description': 'Locus has one or more Alerts with high SNR.',
        },
    ]

    def run(self, locus):
        """
        If this Alert has a high SNR, then tag the Locus "high_snr".
        """
        # The threshold is dependent on the band that is being imaged.
        # These thresholds should flag ~2-3% of alerts.
        snr_threshold = {
            'g': 50.0,
            'R': 55.0,
        }
        passband = locus.alert.properties['ant_passband']
        if passband not in snr_threshold:
            print(f'passband {passband} is not supported by this filter.')
            return  # Do nothing.
        threshold = snr_threshold[passband]
        sigmapsf = locus.alert.properties['ztf_sigmapsf']  # Get the ZTF Alert property "sigmapsf"
        alert_snr = 1.0 / sigmapsf
        alert_id = locus.alert.alert_id  # Get the ANTARES alert_id
        ztf_object_id = locus.properties['ztf_object_id']  # Get the ZTF Object ID
        print(f'Alert {alert_id}')
        print(f'Object {ztf_object_id}')
        print(f'snr = {alert_snr}')
        if alert_snr > threshold:
            print('High SNR detected')
            locus.tag('high_snr')

<a class="anchor" id="write-three"></a>
### 2.3 Structure of a Filter


The filter `MyFilter` below does nothing of scientific interest, but it demonstrates the most basic use of the filter API.

The Filter API consists of the Locus Object, which is passed to the Filter as the single parameter. The `MyFilter` shows examples of how to use the Locus data. For detailed information on the Locus Object, please visit ANTARES documentation at https://noao.gitlab.io/antares/filter-documentation/devkit/locus.html

In [6]:
class MyFilter(dk.Filter):

    # Required.
    #
    # This allows you to receive error logs through Slack.
    # See footnotes below for more details.
    ERROR_SLACK_CHANNEL = '<my_slack_member_id>'

    # Optional.
    #
    # List of Locus properties which the Filter depends on.
    # If an incoming Alert's Locus does not have all properties listed here,
    # then the Filter will not run on it.
    INPUT_LOCUS_PROPERTIES = [
        # eg:
        'ztf_object_id',
        # etc.
    ]

    # Optional.
    #
    # List of Alert properties which the Filter depends on.
    # If an incoming Alert does not have all properties listed here, then
    # the Filter will not run on it.
    INPUT_ALERT_PROPERTIES = [
        # eg:
        'passband',
        'mag',
        'ztf_magpsf',
        # etc.
    ]

    # Optional.
    #
    # List of Tag names which the Filter depends on.
    # If an incoming Alert's Locus does not have all Tags listed here, then
    # the Filter will not run on it.
    INPUT_TAGS = [
        # eg:
        'high_snr',
        # etc.
    ]

    # Required.
    #
    # A list of all Alert properties which the filter may set.
    # If your filter doesn't set properties, then value should be an
    # empty list.
    # 'name' must be formatted like '<author>_<property_name>'.
    # 'type' must be one of the strings: 'int', 'float', or 'str'.
    # 'description' should briefly describe what the property means.
    OUTPUT_LOCUS_PROPERTIES = [
        # eg:
        {
            'name': 'stubens_interest_score',
            'type': 'float',
            'description': 'interestingness of the alert by algorithm XYZ',
        },
        {
            'name': 'stubens_object_class',
            'type': 'str',
            'description': 'probable class of object by algorithm ABC',
        },
        # etc.
    ]

    # Required.
    #
    # A list of all Alert properties which the filter may set.
    # If your filter doesn't set properties, then value should be an
    # empty list.
    # 'name' must be formatted like '<author>_<property_name>'.
    # 'type' must be one of the strings: 'int', 'float', or 'str'.
    # 'description' should briefly describe what the property means.
    OUTPUT_ALERT_PROPERTIES = [
        # eg:
        {
            'name': 'stubens_g_minus_r',
            'type': 'float',
            'description': 'estimated g-minus-r magnitude',
        },
        # etc.
    ]

    # Required.
    #
    # A list tags names which this Filter may produce.
    # If your filter does't tag Loci, then this list should be empty.
    # 'name' must be formatted like '<author>_<property_name>'.
    # 'description' should briefly describe what the tag means.
    OUTPUT_TAGS = [
        # eg:
        {
            'name': 'stubens_transients',
            'description': 'Probable transient according to method PQE'
        },
        # etc.
    ]

    # Optional.
    #
    # If your filter requires access to data files, they must be declared here.
    # See footnotes below for more details on how to work with data files.
    REQUIRES_FILES = [
        # eg:
        'soraisam_myFile.txt',
        'soraisam_myOtherFile.bin',
        # etc.
    ]

    # Optional.
    #
    # This function is called once per night when ANTARES reboots and
    # filters are instantiated. If your filter needs to do any work to prepare
    # itself to run, that logic should go here.
    # Examples:
    #  - Loading data from files
    #  - Constructing datastructures
    #  - Instantiating machine-learning model objects
    def setup(self):
        ...

    # Required.
    #
    # This is the function which is called to process an Alert.
    # All setup work should have been done in `setup()` in order to make this
    # function run as efficiently as possible.
    # See footnotes below for description of the `locus` object.
    def run(self, locus):
        ...



<a class="anchor" id="test"></a>
## 3. Test a filter


Let’s run a simple `HelloWorld` filter:

In [7]:
class HelloWorld(dk.Filter):
    OUTPUT_TAGS = [
        {
            'name': 'hello_world',
            'description': 'hello!',
        },
    ]

    def run(self, locus):
        print('Hello Locus', locus.locus_id)
        locus.tag('hello_world')


We can use `run_filter()` to run the filter on a randomly chose real Locus from the database. We can also ask the filter to run a specific locus by giving a locus ID:

In [8]:
# Execute HelloWorld filter on a random locus
report = dk.run_filter(HelloWorld)

# `run_filter()` returns a report of what the filter did. Take a look at it:
print(report)


# Execute HelloWorld filter on a specified locus
locus_id = 'ANT2020hcm7s'
report = dk.run_filter(HelloWorld, locus=locus_id)

# `run_filter()` returns a report of what the filter did. Take a look at it:
print(report)

Hello Locus ANT2020irq52
{'locus_id': 'ANT2020irq52', 'locus_data': LocusDataAPI(locus_id="ANT2020irq52"), 't': 0.0008711814880371094, 'new_locus_properties': {}, 'new_alert_properties': {}, 'new_tags': ['hello_world'], 'raised_halt': False}
Hello Locus ANT2020hcm7s
{'locus_id': 'ANT2020hcm7s', 'locus_data': LocusDataAPI(locus_id="ANT2020hcm7s"), 't': 0.0006685256958007812, 'new_locus_properties': {}, 'new_alert_properties': {}, 'new_tags': ['hello_world'], 'raised_halt': False}


For testing purpose, we recommend to run the filter on multiple Loci using `run_many()`:


In [9]:
# Execute HelloWorld filter on 100 random locii
report = dk.run_many(HelloWorld, n=100)

# `run_many()` returns a report of what the filter did. Take a look at it:
print(report)

Hello Locus ANT2018bgbn2
Hello Locus ANT2018cq5qo
Hello Locus ANT2020frl24
Hello Locus ANT2020ihls2
Hello Locus ANT2020ln6gg
Hello Locus ANT2018f632u
Hello Locus ANT2019bqwp4
Hello Locus ANT2020cwsmg
Hello Locus ANT2020i7lz4
Hello Locus ANT2020rkr5q
Hello Locus ANT2019djkja
Hello Locus ANT2020q5g34
Hello Locus ANT2020ocz7y
Hello Locus ANT2018h2rqg
Hello Locus ANT2018dsow6
Hello Locus ANT2020onc32
Hello Locus ANT2020e64xm
Hello Locus ANT2020trina
Hello Locus ANT2020q4k3k
Hello Locus ANT2020xe46i
Hello Locus ANT2020nues4
Hello Locus ANT2020f7dzg
Hello Locus ANT2020d2s4o
Hello Locus ANT2018fm57y
Hello Locus ANT2019axatw
Hello Locus ANT2019cnyo6
Hello Locus ANT2018gbahk
Hello Locus ANT2019dxq7e
Hello Locus ANT2020qvjbi
Hello Locus ANT2018c7mkk
Hello Locus ANT2018btpv2
Hello Locus ANT2020hgqsa
Hello Locus ANT2020b2yxo
Hello Locus ANT20202hcos
Hello Locus ANT2020g5nfo
Hello Locus ANT2018evmga
Hello Locus ANT2020as6xs
Hello Locus ANT2020ttopq
Hello Locus ANT2020kk4ls
Hello Locus ANT2018f2jvw


## Constructing Locus Objects
You can also construct your own Locus objects for testing:

In [10]:
ra, dec = 88.2744186, -5.0010774
locus_dict = {
    'locus_id': 'locus1',
    'ra': ra,
    'dec': dec,
    'properties': {
        'num_alerts': 2,
        'num_mag_values': 2,
    },
    'tags': [],
    'watch_list_ids': [],
    'watch_object_ids': [],
    'catalog_objects': dk.search_catalogs(ra, dec),
    'alerts': [
        {
            'alert_id': 'alert1',
            'locus_id': 'locus1',
            'mjd': 58794.272488399874,
            'properties': {
                'ant_mag': 15.1,
            },
        },
        {
            'alert_id': 'alert2',
            'locus_id': 'locus1',
            'mjd': 58799.50587960007,
            'properties': {
                'ant_mag': 15.2,
            }
        },
    ],
}

locus = dk.locus_from_dict(locus_dict)

dk.run_filter(HelloWorld, locus)

Hello Locus locus1


{'locus_id': 'locus1',
 'locus_data': LocusDataAPI(locus_id="locus1"),
 't': 0.00070953369140625,
 'new_locus_properties': {},
 'new_alert_properties': {},
 'new_tags': ['hello_world'],
 'raised_halt': False}

<a class="anchor" id="data"></a>
## 4. Upload and use Data File

Some filters require access to data files, such as statistical models. ANTARES supports this by storing such files as binary blobs in a database table. These data files can then be loaded into filters when the filter’s `setup()` function is called.

### Uploading files into ANTARES

The detail of uploading files into ANTARES can be found at ANTARES [Documentation](https://noao.gitlab.io/antares/filter-documentation/devkit/files.html)

### Accessing Files from Filters
In the `setup()` function of your filter, file data are available as bytes objects.

Here is an example of a filter which requires a file, transforms that file into a Python object or datastructure, and then uses that object when the filter runs:

In [11]:
import antares.devkit as dk


class MyFilter(dk.Filter):
    REQUIRES_FILES = [
        'cstubens_myFile.txt'
    ]

    def setup(self):
        """
        ANTARES will call this function once at the beginning of each night
        when filters are loaded.
        """
        # ANTARES will load all files in `REQUIRED_FILES` into a dict
        # on the filter object `self.files`. Values in this dictionary
        # will be byte strings of class `bytes`.
        # You can then access them like:
        file_data = self.files['cstubens_myFile.txt']

        # Construct a Python object or datastructure from the raw `file_data`.
        # TODO: your code here
        # Then, you can store it on the filter instance for use in `run()`:
        self.my_object = my_object

    def run(self, locus):
        """
        ANTARES will call this function in real-time for each incoming Alert.
        """
        # Here you can use `self.my_object` in your processing of the alert.
        # TODO: your code here

### Filter Submission
BEFORE you submit your filter to ANTARES, you must contact us to request that we copy your datafiles from the DevKit database into the production database. Please provide the file `key` value(s) which you uploaded the files to.

<a class="anchor" id="submit"></a>
## 5. Submit Filter to ANTARES

When you're ready to submit your filter to ANTARES, go to the [filters](https://antares.noirlab.edu/filters) page on the ANTARES website and click "Add".

**Note**: We highly recommend setting an `ERROR_LOG_SLACK_CHANNEL` on your filter so that you will receive notifications of errors. See [Debugging Filters in Production](https://noao.gitlab.io/antares/filter-documentation/devkit/debugging.html#devkit-debugging).

When you submit your filter you will need to provide:

* **Name** – A unique name for your filter. Name your filter like:

     * Format: `<author or group>_<name>`

     * eg: `stubens_sn1a_candidates`
     

* **Description** – A brief text description of your filter. Will be publicly visible.


* **Handler** – The name of the filter class in your code. The handler name does not need to be unique outside of your code.


* **Code** – A block of code which includes:

     * your import statements

     * filter class

     * any helper functions that your filter needs
